# Extracting Relevant Information from Clinical Notes 


## There is much to be learned from unstructured healthcare data, particularly, SOAP notes dictated or written by a provider. Evaluating the text of these notes allows analysis of morbidity, mortality, symptomology and many ontologies of patients and diseases. We may even find that the questions we initiated our journey with the intent of answering lead to even more questions and/or explanations.  

In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re #TEXT column processing

In [2]:
#list files in Kaggle input directory
#import os 
#for dirname, _, filenames in os.walk('/kaggle/input'): 
    #for filename in filenames: 
        #print(os.path.join(dirname, filename))

#clinNotesRaw = pd.read_csv('/kaggle/input/clinical-notes/medical_data.csv')

In [23]:
clinNotesRaw = pd.read_csv('/Volumes/Starbuck/RPythSamps/medical_data.csv')

In [4]:
#examine dataframe
print("The column names are listed below:\n")
clinNotesRaw.info()
clinNotesRaw.columns[clinNotesRaw.isnull().any()].tolist() #generate list of columns w/missing values

The column names are listed below:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   SUBJECT_ID      744 non-null    int64 
 1   ROW_ID          744 non-null    int64 
 2   HADM_ID         744 non-null    int64 
 3   CATEGORY        744 non-null    object
 4   ADMISSION_TYPE  744 non-null    object
 5   DIAGNOSIS       744 non-null    object
 6   TEXT            744 non-null    object
dtypes: int64(3), object(4)
memory usage: 40.8+ KB


[]

### The function below will provide feedback on missing values in the dataset and information regarding the frequency of errors made during data entry and/or created during the ETL process. 

In [24]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("The selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
         #mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_zero_values_table(clinNotesRaw)


The selected dataframe has 7 columns and 744 Rows.
There are 0 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type


### We can also check for duplicate values in the data. As evidenced below, there are no duplicates in the dataframe.

In [25]:
clinNotesRaw.duplicated().sum()

0

### We can view the first 5 rows of the dataframe and determine what column contains the text we are interested in processing since their are no missing values or duplicates in the dataset. 

In [26]:
clinNotesRaw.head(5)

,SUBJECT_ID,ROW_ID,HADM_ID,CATEGORY,ADMISSION_TYPE,DIAGNOSIS,TEXT
0,26880,178,135453,Discharge summary,EMERGENCY,S/P FALL;TELEMETRY,Admission Date: [**2162-3-3**] D...
1,42130,181,114236,Discharge summary,ELECTIVE,LEFT SPHENOID MENENGIOMA/SDA,Admission Date: [**2150-2-25**] ...
2,76874,212,113329,Discharge summary,EMERGENCY,TYLENOL BENZO OVERDOSE,Admission Date: [**2101-10-25**] ...
3,66479,220,134640,Discharge summary,EMERGENCY,PEDISTRIAN STRUCK,Admission Date: [**2148-2-3**] D...
4,31502,8,125483,Discharge summary,EMERGENCY,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,Admission Date: [**2174-5-29**] ...


### Examining the Diagnosis column shows there are multiple diagnosis calls. Let's start here by adding an additional column for the secondary and tertiary diagnoses,and so on.

In [27]:
diagExpanded = clinNotesRaw['DIAGNOSIS'].str.split(';', expand = True)
diagExpanded.columns = ['DIAGNOSIS' + str(i) for i in diagExpanded.columns]

diagExpConcat = pd.concat([clinNotesRaw, diagExpanded], axis = 1)
diagExpConcat.head()

,SUBJECT_ID,ROW_ID,HADM_ID,CATEGORY,ADMISSION_TYPE,DIAGNOSIS,TEXT,DIAGNOSIS0,DIAGNOSIS1,DIAGNOSIS2,DIAGNOSIS3
0,26880,178,135453,Discharge summary,EMERGENCY,S/P FALL;TELEMETRY,Admission Date: [**2162-3-3**] D...,S/P FALL,TELEMETRY,None,None
1,42130,181,114236,Discharge summary,ELECTIVE,LEFT SPHENOID MENENGIOMA/SDA,Admission Date: [**2150-2-25**] ...,LEFT SPHENOID MENENGIOMA/SDA,None,None,None
2,76874,212,113329,Discharge summary,EMERGENCY,TYLENOL BENZO OVERDOSE,Admission Date: [**2101-10-25**] ...,TYLENOL BENZO OVERDOSE,None,None,None
3,66479,220,134640,Discharge summary,EMERGENCY,PEDISTRIAN STRUCK,Admission Date: [**2148-2-3**] D...,PEDISTRIAN STRUCK,None,None,None
4,31502,8,125483,Discharge summary,EMERGENCY,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,Admission Date: [**2174-5-29**] ...,RESPIRATORY FAILURE,CONGESTIVE HEART FAILURE,None,None


### Examining a random observation from the dataset, we can see the significant information that accompanies the transcript of clinical notes.

In [28]:
diagExpConcat.iloc[2, 6] #row 2 column 6 (TEXT) data

"Admission Date:  [**2101-10-25**]              Discharge Date:   [**2101-10-28**]\n\nDate of Birth:  [**2064-10-2**]             Sex:   F\n\nService: MEDICINE\n\nAllergies:\nCephalosporins / Floxin / Penicillins\n\nAttending:[**First Name3 (LF) 2108**]\nChief Complaint:\nXanax, Tylenol & Klonopin Overdose\n\nMajor Surgical or Invasive Procedure:\nNone\n\nHistory of Present Illness:\n37 yo F with history of depression and suicidal attempt in the\npast presented with obtundation.  Of note, her prior attempt was\nabout 15 years ago during which she OD on theophylline,\nrequiring intubation.  She has been feeling more depressed over\nthe last few months and has been seeing a therapist, on the ECT\nwaiting list with recent evaluation by Dr. [**Last Name (STitle) 2109**], [**First Name3 (LF) **] her\npartner.  [**Name (NI) **] reports taking 120 mg of Xanax and 80 mg\nKlonopin in the afternoon of [**2101-10-25**] as well as at least [**4-7**]\ng of Tylenol daily over the last 2 weeks.  She 

### Let's attempt to divide the clinical notes transcript.

In [29]:
ntsExpanded = diagExpConcat['TEXT'].str.split('\n\n', expand = True)
ntsExpanded.columns = ['NOTES' + str(i) for i in ntsExpanded.columns]

ntsExpConcat = pd.concat([diagExpConcat, ntsExpanded], axis = 1)
ntsExpConcat.head()

,SUBJECT_ID,ROW_ID,HADM_ID,CATEGORY,ADMISSION_TYPE,DIAGNOSIS,TEXT,DIAGNOSIS0,DIAGNOSIS1,DIAGNOSIS2,...,NOTES126,NOTES127,NOTES128,NOTES129,NOTES130,NOTES131,NOTES132,NOTES133,NOTES134,NOTES135
0,26880,178,135453,Discharge summary,EMERGENCY,S/P FALL;TELEMETRY,Admission Date: [**2162-3-3**] D...,S/P FALL,TELEMETRY,None,...,None,None,None,None,None,None,None,None,None,None
1,42130,181,114236,Discharge summary,ELECTIVE,LEFT SPHENOID MENENGIOMA/SDA,Admission Date: [**2150-2-25**] ...,LEFT SPHENOID MENENGIOMA/SDA,None,None,...,None,None,None,None,None,None,None,None,None,None
2,76874,212,113329,Discharge summary,EMERGENCY,TYLENOL BENZO OVERDOSE,Admission Date: [**2101-10-25**] ...,TYLENOL BENZO OVERDOSE,None,None,...,None,None,None,None,None,None,None,None,None,None
3,66479,220,134640,Discharge summary,EMERGENCY,PEDISTRIAN STRUCK,Admission Date: [**2148-2-3**] D...,PEDISTRIAN STRUCK,None,None,...,None,None,None,None,None,None,None,None,None,None
4,31502,8,125483,Discharge summary,EMERGENCY,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,Admission Date: [**2174-5-29**] ...,RESPIRATORY FAILURE,CONGESTIVE HEART FAILURE,None,...,None,None,None,None,None,None,None,None,None,None


### As evidenced above splitting the column with the same approach didn't split all of the columns and it created 135 columns, most of which have None values. Maybe we can put the dates in a single column and the actual clinical notes in their own column.

In [30]:
ntsExpanded1 = diagExpConcat['TEXT'].str.split('Sex:', expand = True)
ntsExpanded1.columns = ['UNPROCESSED_NOTES' + str(i) for i in ntsExpanded1.columns]

ntsExpConcat1 = pd.concat([diagExpConcat, ntsExpanded1], axis = 1)
ntsExpConcat1.head()

,SUBJECT_ID,ROW_ID,HADM_ID,CATEGORY,ADMISSION_TYPE,DIAGNOSIS,TEXT,DIAGNOSIS0,DIAGNOSIS1,DIAGNOSIS2,DIAGNOSIS3,UNPROCESSED_NOTES0,UNPROCESSED_NOTES1
0,26880,178,135453,Discharge summary,EMERGENCY,S/P FALL;TELEMETRY,Admission Date: [**2162-3-3**] D...,S/P FALL,TELEMETRY,None,None,Admission Date: [**2162-3-3**] D...,M\n\nService: MEDICINE\n\nAllergies:\nPatie...
1,42130,181,114236,Discharge summary,ELECTIVE,LEFT SPHENOID MENENGIOMA/SDA,Admission Date: [**2150-2-25**] ...,LEFT SPHENOID MENENGIOMA/SDA,None,None,None,Admission Date: [**2150-2-25**] ...,M\n\nService: NEUROSURGERY\n\nAllergies:\nN...
2,76874,212,113329,Discharge summary,EMERGENCY,TYLENOL BENZO OVERDOSE,Admission Date: [**2101-10-25**] ...,TYLENOL BENZO OVERDOSE,None,None,None,Admission Date: [**2101-10-25**] ...,F\n\nService: MEDICINE\n\nAllergies:\nCepha...
3,66479,220,134640,Discharge summary,EMERGENCY,PEDISTRIAN STRUCK,Admission Date: [**2148-2-3**] D...,PEDISTRIAN STRUCK,None,None,None,Admission Date: [**2148-2-3**] D...,F\n\nService: SURGERY\n\nAllergies:\nNo Kno...
4,31502,8,125483,Discharge summary,EMERGENCY,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,Admission Date: [**2174-5-29**] ...,RESPIRATORY FAILURE,CONGESTIVE HEART FAILURE,None,None,Admission Date: [**2174-5-29**] ...,F\n\nService: MEDICINE\n\nAllergies:\nAtorv...


In [31]:
ntsExpConcat1.iloc[2, 11] 

'Admission Date:  [**2101-10-25**]              Discharge Date:   [**2101-10-28**]\n\nDate of Birth:  [**2064-10-2**]             '

### Eureka! We have what appears to be a successful extraction of the Admission and Discharge Dates. As a bonus we were also able to extract the date of birth. Let's go ahead and give them their own, separate columns for ease of access. 

In [32]:
datesExpanded = ntsExpConcat1['UNPROCESSED_NOTES0'].str.split('Discharge Date:', expand =True)

datesExpanded.columns = ['ADMISSION_DATE' + str(i) for i in datesExpanded.columns]

datesExpConcat = pd.concat([ntsExpConcat1, datesExpanded], axis = 1)
datesExpConcat.head()

,SUBJECT_ID,ROW_ID,HADM_ID,CATEGORY,ADMISSION_TYPE,DIAGNOSIS,TEXT,DIAGNOSIS0,DIAGNOSIS1,DIAGNOSIS2,DIAGNOSIS3,UNPROCESSED_NOTES0,UNPROCESSED_NOTES1,ADMISSION_DATE0,ADMISSION_DATE1
0,26880,178,135453,Discharge summary,EMERGENCY,S/P FALL;TELEMETRY,Admission Date: [**2162-3-3**] D...,S/P FALL,TELEMETRY,None,None,Admission Date: [**2162-3-3**] D...,M\n\nService: MEDICINE\n\nAllergies:\nPatie...,Admission Date: [**2162-3-3**],[**2162-3-25**]\n\nDate of Birth: [**2080-...
1,42130,181,114236,Discharge summary,ELECTIVE,LEFT SPHENOID MENENGIOMA/SDA,Admission Date: [**2150-2-25**] ...,LEFT SPHENOID MENENGIOMA/SDA,None,None,None,Admission Date: [**2150-2-25**] ...,M\n\nService: NEUROSURGERY\n\nAllergies:\nN...,Admission Date: [**2150-2-25**],[**2150-3-1**]\n\nDate of Birth: [**2086-1...
2,76874,212,113329,Discharge summary,EMERGENCY,TYLENOL BENZO OVERDOSE,Admission Date: [**2101-10-25**] ...,TYLENOL BENZO OVERDOSE,None,None,None,Admission Date: [**2101-10-25**] ...,F\n\nService: MEDICINE\n\nAllergies:\nCepha...,Admission Date: [**2101-10-25**],[**2101-10-28**]\n\nDate of Birth: [**2064...
3,66479,220,134640,Discharge summary,EMERGENCY,PEDISTRIAN STRUCK,Admission Date: [**2148-2-3**] D...,PEDISTRIAN STRUCK,None,None,None,Admission Date: [**2148-2-3**] D...,F\n\nService: SURGERY\n\nAllergies:\nNo Kno...,Admission Date: [**2148-2-3**],[**2148-2-7**]\n\nDate of Birth: [**2087-6...
4,31502,8,125483,Discharge summary,EMERGENCY,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,Admission Date: [**2174-5-29**] ...,RESPIRATORY FAILURE,CONGESTIVE HEART FAILURE,None,None,Admission Date: [**2174-5-29**] ...,F\n\nService: MEDICINE\n\nAllergies:\nAtorv...,Admission Date: [**2174-5-29**],[**2174-6-9**]\n\nDate of Birth: [**2093-1...


In [33]:
datesExpConcat.iloc[2, 11]

'Admission Date:  [**2101-10-25**]              Discharge Date:   [**2101-10-28**]\n\nDate of Birth:  [**2064-10-2**]             '

In [34]:
#Create a dataframe of ADMISSION_DATE column
newAdmits = datesExpConcat[['ADMISSION_DATE0']].copy()

print("Referring to our initial import of the dataset we confirm that the intial dataset has ", len(clinNotesRaw.index), "rows. \nThe new dataset, newAdmits, has ", len(newAdmits), "rows.")

Referring to our initial import of the dataset we confirm that the intial dataset has  744 rows. 
The new dataset, newAdmits, has  744 rows.


### We can manipulate the new dataframe in order to clean up the data so that it will be easier to format in the future. The years have been altered for patient privacy so I will not be implementing any date/time formatting.

In [35]:
print(newAdmits.dtypes)
newAdmits.rename(columns = {'ADMISSION_DATE0': 'ADMISSION_DATE'}, inplace = True)
newAdmitsClean = newAdmits['ADMISSION_DATE'].str.replace(r'[*, Admission Date:, \([\]]+', '').str.strip()
newAdmitsClean1 = pd.concat([datesExpConcat, newAdmitsClean], axis = 1)
newAdmitsClean1.head()

ADMISSION_DATE0    object
dtype: object


,SUBJECT_ID,ROW_ID,HADM_ID,CATEGORY,ADMISSION_TYPE,DIAGNOSIS,TEXT,DIAGNOSIS0,DIAGNOSIS1,DIAGNOSIS2,DIAGNOSIS3,UNPROCESSED_NOTES0,UNPROCESSED_NOTES1,ADMISSION_DATE0,ADMISSION_DATE1,ADMISSION_DATE
0,26880,178,135453,Discharge summary,EMERGENCY,S/P FALL;TELEMETRY,Admission Date: [**2162-3-3**] D...,S/P FALL,TELEMETRY,None,None,Admission Date: [**2162-3-3**] D...,M\n\nService: MEDICINE\n\nAllergies:\nPatie...,Admission Date: [**2162-3-3**],[**2162-3-25**]\n\nDate of Birth: [**2080-...,2162-3-3
1,42130,181,114236,Discharge summary,ELECTIVE,LEFT SPHENOID MENENGIOMA/SDA,Admission Date: [**2150-2-25**] ...,LEFT SPHENOID MENENGIOMA/SDA,None,None,None,Admission Date: [**2150-2-25**] ...,M\n\nService: NEUROSURGERY\n\nAllergies:\nN...,Admission Date: [**2150-2-25**],[**2150-3-1**]\n\nDate of Birth: [**2086-1...,2150-2-25
2,76874,212,113329,Discharge summary,EMERGENCY,TYLENOL BENZO OVERDOSE,Admission Date: [**2101-10-25**] ...,TYLENOL BENZO OVERDOSE,None,None,None,Admission Date: [**2101-10-25**] ...,F\n\nService: MEDICINE\n\nAllergies:\nCepha...,Admission Date: [**2101-10-25**],[**2101-10-28**]\n\nDate of Birth: [**2064...,2101-10-25
3,66479,220,134640,Discharge summary,EMERGENCY,PEDISTRIAN STRUCK,Admission Date: [**2148-2-3**] D...,PEDISTRIAN STRUCK,None,None,None,Admission Date: [**2148-2-3**] D...,F\n\nService: SURGERY\n\nAllergies:\nNo Kno...,Admission Date: [**2148-2-3**],[**2148-2-7**]\n\nDate of Birth: [**2087-6...,2148-2-3
4,31502,8,125483,Discharge summary,EMERGENCY,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,Admission Date: [**2174-5-29**] ...,RESPIRATORY FAILURE,CONGESTIVE HEART FAILURE,None,None,Admission Date: [**2174-5-29**] ...,F\n\nService: MEDICINE\n\nAllergies:\nAtorv...,Admission Date: [**2174-5-29**],[**2174-6-9**]\n\nDate of Birth: [**2093-1...,2174-5-29


In [68]:
#convert to dataframe
print(newAdmitsClean1.isnull().sum())
print(newAdmitsClean1.isna().sum())

SUBJECT_ID              0
ROW_ID                  0
HADM_ID                 0
CATEGORY                0
ADMISSION_TYPE          0
DIAGNOSIS               0
TEXT                    0
DIAGNOSIS0              0
DIAGNOSIS1            642
DIAGNOSIS2            722
DIAGNOSIS3            743
UNPROCESSED_NOTES0      0
UNPROCESSED_NOTES1    115
ADMISSION_DATE0         0
ADMISSION_DATE1         2
ADMISSION_DATE          0
dtype: int64
SUBJECT_ID              0
ROW_ID                  0
HADM_ID                 0
CATEGORY                0
ADMISSION_TYPE          0
DIAGNOSIS               0
TEXT                    0
DIAGNOSIS0              0
DIAGNOSIS1            642
DIAGNOSIS2            722
DIAGNOSIS3            743
UNPROCESSED_NOTES0      0
UNPROCESSED_NOTES1    115
ADMISSION_DATE0         0
ADMISSION_DATE1         2
ADMISSION_DATE          0
dtype: int64


### There appears to be 2 observations for the discharge date that have NA values. This may be due to formatting. Data cleaning practices similar to those employed above when working with the ADMISSION_DATE column could be used to correct this. The data can be corrected once the location is found. I'm going to leave the dates as is and continue forward.

In [36]:
#find the 2 NA values using index
#newAdmitsClean1.loc[pd.isna(newAdmitsClean1['ADMISSION_DATE1']), :].index

In [43]:
#perform the same extraction for the discharge dates
newDischarges = datesExpConcat[['ADMISSION_DATE1']].copy()
print("New dataset obseverations: ", len(newDischarges)) 

#split DOB and Discharge date
newDischarges1 = newDischarges['ADMISSION_DATE1'].str.split('Date of Birth:', expand = True)
newDischarges1 #change column 0 name to disharge date


New dataset obseverations:  744


,0,1
0,[**2162-3-25**]\n\n,[**2080-1-4**]
1,[**2150-3-1**]\n\n,[**2086-12-19**]
2,[**2101-10-28**]\n\n,[**2064-10-2**]
3,[**2148-2-7**]\n\n,[**2087-6-7**]
4,[**2174-6-9**]\n\n,[**2093-11-17**]
...,...,...
739,[**2196-12-15**]\n\n\nService: MEDICINE\n\n...,None
740,[**2180-4-9**]\n\n,[**2106-5-7**]
741,[**2165-7-25**]\n\n,[**2081-2-27**]
742,[**2167-9-8**]\n\n,[**2085-1-14**]


### A quick comparison of the datset generated and the query of the TEXT column done before shows that column 0 has the discharge dates and column 1 contains the date of birth data.

In [59]:
newDischargesClean = pd.DataFrame(newDischarges1[0].str.replace(r'[*, \n\n,\([\]]+', ''))
newDischargesClean.info()
newDischargesClean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       742 non-null    object
dtypes: object(1)
memory usage: 5.9+ KB


,0
0,2162-3-25
1,2150-3-1
2,2101-10-28
3,2148-2-7
4,2174-6-9


In [60]:
newDischargesClean.rename(columns = {0: 'DISCHARGE_DATE'}, inplace= True)
newDischargesClean.head()

,DISCHARGE_DATE
0,2162-3-25
1,2150-3-1
2,2101-10-28
3,2148-2-7
4,2174-6-9


In [69]:
admitDischarge = pd.concat([newAdmitsClean1, newDischargesClean], axis = 1)
admitDischarge.head()

,SUBJECT_ID,ROW_ID,HADM_ID,CATEGORY,ADMISSION_TYPE,DIAGNOSIS,TEXT,DIAGNOSIS0,DIAGNOSIS1,DIAGNOSIS2,DIAGNOSIS3,UNPROCESSED_NOTES0,UNPROCESSED_NOTES1,ADMISSION_DATE0,ADMISSION_DATE1,ADMISSION_DATE,DISCHARGE_DATE
0,26880,178,135453,Discharge summary,EMERGENCY,S/P FALL;TELEMETRY,Admission Date: [**2162-3-3**] D...,S/P FALL,TELEMETRY,None,None,Admission Date: [**2162-3-3**] D...,M\n\nService: MEDICINE\n\nAllergies:\nPatie...,Admission Date: [**2162-3-3**],[**2162-3-25**]\n\nDate of Birth: [**2080-...,2162-3-3,2162-3-25
1,42130,181,114236,Discharge summary,ELECTIVE,LEFT SPHENOID MENENGIOMA/SDA,Admission Date: [**2150-2-25**] ...,LEFT SPHENOID MENENGIOMA/SDA,None,None,None,Admission Date: [**2150-2-25**] ...,M\n\nService: NEUROSURGERY\n\nAllergies:\nN...,Admission Date: [**2150-2-25**],[**2150-3-1**]\n\nDate of Birth: [**2086-1...,2150-2-25,2150-3-1
2,76874,212,113329,Discharge summary,EMERGENCY,TYLENOL BENZO OVERDOSE,Admission Date: [**2101-10-25**] ...,TYLENOL BENZO OVERDOSE,None,None,None,Admission Date: [**2101-10-25**] ...,F\n\nService: MEDICINE\n\nAllergies:\nCepha...,Admission Date: [**2101-10-25**],[**2101-10-28**]\n\nDate of Birth: [**2064...,2101-10-25,2101-10-28
3,66479,220,134640,Discharge summary,EMERGENCY,PEDISTRIAN STRUCK,Admission Date: [**2148-2-3**] D...,PEDISTRIAN STRUCK,None,None,None,Admission Date: [**2148-2-3**] D...,F\n\nService: SURGERY\n\nAllergies:\nNo Kno...,Admission Date: [**2148-2-3**],[**2148-2-7**]\n\nDate of Birth: [**2087-6...,2148-2-3,2148-2-7
4,31502,8,125483,Discharge summary,EMERGENCY,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,Admission Date: [**2174-5-29**] ...,RESPIRATORY FAILURE,CONGESTIVE HEART FAILURE,None,None,Admission Date: [**2174-5-29**] ...,F\n\nService: MEDICINE\n\nAllergies:\nAtorv...,Admission Date: [**2174-5-29**],[**2174-6-9**]\n\nDate of Birth: [**2093-1...,2174-5-29,2174-6-9


In [65]:
dOB = pd.DataFrame(newDischarges1[1].str.replace(r'[*,\([\]]+', ''))
dOB.head()

,1
0,2080-1-4
1,2086-12-19
2,2064-10-2
3,2087-6-7
4,2093-11-17


In [66]:
dOB.rename(columns = {1: 'DOB'}, inplace= True)
dOB.head()

,DOB
0,2080-1-4
1,2086-12-19
2,2064-10-2
3,2087-6-7
4,2093-11-17


In [71]:
#add the new dataframe to the complete dataframe and drop all Admissions columns
adDisDOB = pd.concat([admitDischarge, dOB], axis = 1)
adDisDOB.head()

,SUBJECT_ID,ROW_ID,HADM_ID,CATEGORY,ADMISSION_TYPE,DIAGNOSIS,TEXT,DIAGNOSIS0,DIAGNOSIS1,DIAGNOSIS2,DIAGNOSIS3,UNPROCESSED_NOTES0,UNPROCESSED_NOTES1,ADMISSION_DATE0,ADMISSION_DATE1,ADMISSION_DATE,DISCHARGE_DATE,DOB
0,26880,178,135453,Discharge summary,EMERGENCY,S/P FALL;TELEMETRY,Admission Date: [**2162-3-3**] D...,S/P FALL,TELEMETRY,None,None,Admission Date: [**2162-3-3**] D...,M\n\nService: MEDICINE\n\nAllergies:\nPatie...,Admission Date: [**2162-3-3**],[**2162-3-25**]\n\nDate of Birth: [**2080-...,2162-3-3,2162-3-25,2080-1-4
1,42130,181,114236,Discharge summary,ELECTIVE,LEFT SPHENOID MENENGIOMA/SDA,Admission Date: [**2150-2-25**] ...,LEFT SPHENOID MENENGIOMA/SDA,None,None,None,Admission Date: [**2150-2-25**] ...,M\n\nService: NEUROSURGERY\n\nAllergies:\nN...,Admission Date: [**2150-2-25**],[**2150-3-1**]\n\nDate of Birth: [**2086-1...,2150-2-25,2150-3-1,2086-12-19
2,76874,212,113329,Discharge summary,EMERGENCY,TYLENOL BENZO OVERDOSE,Admission Date: [**2101-10-25**] ...,TYLENOL BENZO OVERDOSE,None,None,None,Admission Date: [**2101-10-25**] ...,F\n\nService: MEDICINE\n\nAllergies:\nCepha...,Admission Date: [**2101-10-25**],[**2101-10-28**]\n\nDate of Birth: [**2064...,2101-10-25,2101-10-28,2064-10-2
3,66479,220,134640,Discharge summary,EMERGENCY,PEDISTRIAN STRUCK,Admission Date: [**2148-2-3**] D...,PEDISTRIAN STRUCK,None,None,None,Admission Date: [**2148-2-3**] D...,F\n\nService: SURGERY\n\nAllergies:\nNo Kno...,Admission Date: [**2148-2-3**],[**2148-2-7**]\n\nDate of Birth: [**2087-6...,2148-2-3,2148-2-7,2087-6-7
4,31502,8,125483,Discharge summary,EMERGENCY,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,Admission Date: [**2174-5-29**] ...,RESPIRATORY FAILURE,CONGESTIVE HEART FAILURE,None,None,Admission Date: [**2174-5-29**] ...,F\n\nService: MEDICINE\n\nAllergies:\nAtorv...,Admission Date: [**2174-5-29**],[**2174-6-9**]\n\nDate of Birth: [**2093-1...,2174-5-29,2174-6-9,2093-11-17


In [74]:
#drop the columns related to date that are no longer needed
updatedDatesCN = adDisDOB.drop(['ADMISSION_DATE1', 'ADMISSION_DATE0', 'DIAGNOSIS3'], axis = 1)
updatedDatesCN.head()

,SUBJECT_ID,ROW_ID,HADM_ID,CATEGORY,ADMISSION_TYPE,DIAGNOSIS,TEXT,DIAGNOSIS0,DIAGNOSIS1,DIAGNOSIS2,UNPROCESSED_NOTES0,UNPROCESSED_NOTES1,ADMISSION_DATE,DISCHARGE_DATE,DOB
0,26880,178,135453,Discharge summary,EMERGENCY,S/P FALL;TELEMETRY,Admission Date: [**2162-3-3**] D...,S/P FALL,TELEMETRY,None,Admission Date: [**2162-3-3**] D...,M\n\nService: MEDICINE\n\nAllergies:\nPatie...,2162-3-3,2162-3-25,2080-1-4
1,42130,181,114236,Discharge summary,ELECTIVE,LEFT SPHENOID MENENGIOMA/SDA,Admission Date: [**2150-2-25**] ...,LEFT SPHENOID MENENGIOMA/SDA,None,None,Admission Date: [**2150-2-25**] ...,M\n\nService: NEUROSURGERY\n\nAllergies:\nN...,2150-2-25,2150-3-1,2086-12-19
2,76874,212,113329,Discharge summary,EMERGENCY,TYLENOL BENZO OVERDOSE,Admission Date: [**2101-10-25**] ...,TYLENOL BENZO OVERDOSE,None,None,Admission Date: [**2101-10-25**] ...,F\n\nService: MEDICINE\n\nAllergies:\nCepha...,2101-10-25,2101-10-28,2064-10-2
3,66479,220,134640,Discharge summary,EMERGENCY,PEDISTRIAN STRUCK,Admission Date: [**2148-2-3**] D...,PEDISTRIAN STRUCK,None,None,Admission Date: [**2148-2-3**] D...,F\n\nService: SURGERY\n\nAllergies:\nNo Kno...,2148-2-3,2148-2-7,2087-6-7
4,31502,8,125483,Discharge summary,EMERGENCY,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,Admission Date: [**2174-5-29**] ...,RESPIRATORY FAILURE,CONGESTIVE HEART FAILURE,None,Admission Date: [**2174-5-29**] ...,F\n\nService: MEDICINE\n\nAllergies:\nAtorv...,2174-5-29,2174-6-9,2093-11-17


### We can try and make sense of an observation in the TEXT column by separating the data by section of the clinical notes. Scanning the clinical notes, I can see a pattern that remains consistent for the available observations. Similar tactics can be used to split the string and add data to new columns i.e. creating columns for family history, chief complaints, etc.